Based on **Graph Based Approaches for Image Segmentation and Object Tracking**, Xiaofang Wang, September 2014.



Imports

In [1]:
import pandas as pd
import numpy as np

## Directed Acyclic Graph Construction

+ $D = (V,A)$,

+ $V = F_1 \cup F_2 \ldots F_n$, where $n$ is a number of frames,

+ $(u,v) \in A \iff u \in F_l,\ v \in F_{l+1}$ for some $l \in \{1, \ldots, n-1\}$ and $dist(u,v) \leq d_{max}$ (we consider Euclidean distance since $u = (x_1, y_1)$ and $v = (x_2, y_2)$ - coordinates of segments' centroids).

Possible situations:
1. $v \in F_k$ has no neighbour in $F_{k+1}$,
2. $v \in F_k$ has exactly one neighbour in $F_{k+1}$,
3. $v \in F_k$ has more than one neighbour in $F_{k+1}$,
4. $v, u \in F_k,\ v \neq u$ has the same neighbour $w \in F_{k+1}$,

In [2]:
# TODO
# miejsce na schemat - rysunek z kolejnymi frame'ami, segmentami i połączeniami między nimi

Hyperparameters

In [3]:
# d_max = ? # dist max -> dlugosc przekatnej frame'a?
# number_of_frames = ?

Utils

In [4]:
def get_arc_weight(node1, node2, d_max):
    # node = (x,y)
    dist = np.linalg.norm(node1 - node2)
    return dist / d_max

## Graph refinement

Methods:
1. **Configuration I (PI)** - case of missing connection - this can be easily fixed by simply increasing the search range until a neighborhood is found.


2. **Configuration II (PII)** - case where one node connects to two neighbors which are far from each other.  Link with the most distant node is removed from the graph. Let us emphasize that in case a node has more than three neighbors, only the most distant onis removed.


3. **Configuration III (PIII)** - another conflicting situation where two nodes are connected to the same neighbor in $F_{k+1}$. If only one of the node has more than two neighbors then its connection with the common neighbor is removed. If both nodes have more than two neighbors, the longest connection with the common neighbor is removed.

Notice that after these procedures we get a graph that still can have redundant connections. It is shown that repeating above steps of reduction does not improve the result significantly.

## Full Trajectories via Min-cost/max Flow

We expand directed graph $D$ in the following manner:
+ $V := V \cup \{s, t\}$,
+ every arc $(u,v)$ has unit capacity $c$,
+ $f_{uv}$ is a volume of flow from $u$ to $v$ along $e_{uv}$, $f_{uv} \in [0,1]$ or $f_{uv} \in [-1,0]$, when $(u,v)$ is negatively oriented,
+ $w_{uv}$ is a cost per unit flow on $e_{uv}$, namely: $w_{uv} = dist(u,v) / d_max$,
+ edges from $s$ or to $t$ has no cost,
+ sum of inflow and outflow volumes are equal at each node except $s$ and $t$.

Sending a volume $m$ of a circulation flow from $s$ to $t$, and looking for the flow
repartition along the graph with minimal cost is equivalent to solving the following
problem [Ahuja et al., 1993]:

<center>argmin$_f \sum_{(u,v) \in A} w_{uv} f_{uv}$</center>